# CYB80001 System Security Project
Prepared by **Derui (Derek) Wang**

---

# Session 3A - Deep Feedforward Neural Networks with Keras

**The purpose of this session is to demonstrate how to use TensorFlow's implementation of the Keras api to develop machine learning algorithms and deep neural network models. **



## <span style="color:#0b486b">Classification with Keras</span>

Let's repeat the Week 6 practical with Keras and build a simple softmax classification with MNIST dataset.

* #### <span style="color:#0b486b">Step 1: Load or download the dataset</span>

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("datasets/")

C:\Users\thewe\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting datasets/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting datasets/train-labels-idx1-ubyte.gz
Extracting datasets/t10k-images-idx3-ubyte.gz
Extracting datasets/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


* #### <span style="color:#0b486b">Step 2: Build a sequential model</span>


In Week 6, you saw the following TensorFlow code to construct a graph.

```
import tensorflow as tf
tf.reset_default_graph()

num_inputs = 28 * 28
num_hidden1 = 300
num_hidden2 = 100
num_outputs = 10
learning_rate = 0.01


x = tf.placeholder(tf.float32, shape=(None, num_inputs), name="x")
y = tf.placeholder(tf.int64, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(x, num_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, num_hidden2, name="hidden2", activation=tf.nn.relu)
    logits = tf.layers.dense(hidden2, num_outputs, name="outputs")
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    grads = optimizer.compute_gradients(loss)    
    training_op = optimizer.apply_gradients(grads)    
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))   
    
init = tf.global_variables_initializer()
```

Below shows the Keras code to define such a model.

In [2]:
import tensorflow as tf
from tensorflow import keras

n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10
learning_rate = 0.01

model = keras.Sequential()

# Adds a densely-connected layer with n_hidden1 units to the model:
model.add(keras.layers.Dense(n_hidden1, activation='relu'))

# Add another densely-connected layer with n_hidden2 units :
model.add(keras.layers.Dense(n_hidden2, activation='relu'))

# Add a softmax layer with n_outputs output units:
model.add(keras.layers.Dense(n_outputs, activation='softmax'))


# Configure the learning process, including specifying the loss the evaluation metrics
model.compile(optimizer=tf.train.GradientDescentOptimizer(learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=['sparse_categorical_accuracy'])

* #### <span style="color:#0b486b">Step 3: Train and test the model</span>


Similarly the following code for training can be replaced.
``` 
num_epochs = 20
batch_size = 50

with tf.Session() as sess:
    init.run()
    print("Epoch\tTrain accuracy\tTest accuracy")
    for epoch in range(num_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            x_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={x: x_batch, y: y_batch})
            
        acc_train = sess.run(accuracy,
                             feed_dict={x: x_batch, y: y_batch})
        
        acc_test = sess.run(accuracy,
                             feed_dict={x: mnist.test.images, y: mnist.test.labels})
        
        print("{}\t{}\t{}".format(epoch, acc_train))   
``` 

The training is simple with Keras.

In [5]:
num_epochs = 20
batch_size = 50

model.fit(mnist.train.images, 
          mnist.train.labels, 
          epochs=num_epochs, 
          batch_size=batch_size, 
          validation_data=( mnist.test.images, mnist.test.labels))

Train on 55000 samples, validate on 10000 samples
Epoch 1/20
55000/55000 [==============================] - 7s 129us/step - loss: 0.7463 - sparse_categorical_accuracy: 0.8106 - val_loss: 0.3618 - val_sparse_categorical_accuracy: 0.9019
Epoch 2/20
55000/55000 [==============================] - 7s 123us/step - loss: 0.3377 - sparse_categorical_accuracy: 0.9042 - val_loss: 0.2873 - val_sparse_categorical_accuracy: 0.9194
Epoch 3/20
55000/55000 [==============================] - 7s 119us/step - loss: 0.2819 - sparse_categorical_accuracy: 0.9196 - val_loss: 0.2508 - val_sparse_categorical_accuracy: 0.9305
Epoch 4/20
55000/55000 [==============================] - 6s 117us/step - loss: 0.2466 - sparse_categorical_accuracy: 0.9287 - val_loss: 0.2276 - val_sparse_categorical_accuracy: 0.9364
Epoch 5/20
55000/55000 [==============================] - 6s 117us/step - loss: 0.2197 - sparse_categorical_accuracy: 0.9383 - val_loss: 0.2090 - val_sparse_categorical_accuracy: 0.9399
Epoch 6/20
55000/550

An added benefit of using Keras is that the `model` Python variable preserves all the information regarding the model, including the current weights. So we can easily further improve the model with additional training epochs.

In [7]:
history = model.fit(mnist.train.images, 
                    mnist.train.labels,
                    epochs=5,
                    batch_size=batch_size, 
                    validation_data=( mnist.test.images, mnist.test.labels))

Train on 55000 samples, validate on 10000 samples
Epoch 1/5
55000/55000 [==============================] - 8s 151us/step - loss: 0.0535 - sparse_categorical_accuracy: 0.9853 - val_loss: 0.0791 - val_sparse_categorical_accuracy: 0.9764
Epoch 2/5
55000/55000 [==============================] - 9s 164us/step - loss: 0.0501 - sparse_categorical_accuracy: 0.9867 - val_loss: 0.0798 - val_sparse_categorical_accuracy: 0.9760
Epoch 3/5
55000/55000 [==============================] - 9s 166us/step - loss: 0.0468 - sparse_categorical_accuracy: 0.9873 - val_loss: 0.0848 - val_sparse_categorical_accuracy: 0.9742
Epoch 4/5
55000/55000 [==============================] - 9s 165us/step - loss: 0.0442 - sparse_categorical_accuracy: 0.9885 - val_loss: 0.0774 - val_sparse_categorical_accuracy: 0.9770
Epoch 5/5
55000/55000 [==============================] - 9s 163us/step - loss: 0.0414 - sparse_categorical_accuracy: 0.9892 - val_loss: 0.0768 - val_sparse_categorical_accuracy: 0.9767


### Excercises


- Change five different value for the number of hidden nodes in each layer and report the best numbers among your chosen number.
- Increase the number of hidden layers to **three** and set five values for the number of hidden nodes then report the best value and its performance.
- Try to change the optimizer to train the model to [Adam](https://www.tensorflow.org/api_docs/python/tf/train/AdamOptimizer) and [RMSProp](https://www.tensorflow.org/api_docs/python/tf/train/RMSPropOptimizer)

---
### <div  style="text-align:center">**THE END**</div>